# PyTorch Fashion MNIST example

"Hello world" type example, based on https://pytorch.org/tutorials/beginner/basics/optimization_tutorial.html

In [1]:
from dotmap import DotMap
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import torchvision
from tqdm import tqdm

In [2]:
# print versions of imported libraries if can
import sys

for module_name in set(sys.modules) & set(globals()):
    if version := getattr(sys.modules[module_name], "__version__", None):
        print(module_name, version)

tqdm 4.64.1
torch 1.12.1
torchvision 0.13.1


In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"{device = }")
cf = DotMap(
    {
        "learning_rate": 1e-3,
        "batch_size": 64,
        "num_epochs": 5,
    }
)

device = 'cuda'


In [4]:
training_data = torchvision.datasets.FashionMNIST(
    root="data", train=True, download=True, transform=torchvision.transforms.ToTensor()
)
test_data = torchvision.datasets.FashionMNIST(
    root="data", train=False, download=True, transform=torchvision.transforms.ToTensor()
)
train_dataloader = DataLoader(training_data, batch_size=cf.batch_size)
test_dataloader = DataLoader(test_data, batch_size=cf.batch_size)

In [5]:
class Model(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28 * 28, 512),
            nn.LeakyReLU(0.1),
            nn.Linear(512, 512),
            nn.LeakyReLU(0.1),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits


def train_loop(dataloader, model, loss_fn, optimizer, epoch):
    pbar = tqdm(dataloader, desc=f"Epoch {epoch}")
    for batch, (X, y) in enumerate(pbar):
        X, y = X.to(device), y.to(device)
        pred = model(X)
        loss = loss_fn(pred, y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        pbar.set_postfix({"loss": f"{loss.item():.4f}"})


def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test set: Accuracy = {(100*correct):.1f}%, Mean loss = {test_loss:.4f}")

In [6]:
# Train for `cf.num_epochs` epochs
model = Model().to(device)
loss_fn = nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=cf.learning_rate)
for i in range(1, cf.num_epochs + 1):
    train_loop(train_dataloader, model, loss_fn, optimizer, epoch=i)
    test_loop(test_dataloader, model, loss_fn)

Epoch 1: 100%|██████████| 938/938 [00:08<00:00, 105.89it/s, loss=2.1377]


Test set: Accuracy = 43.6%, Mean loss = 2.1484


Epoch 2: 100%|██████████| 938/938 [00:07<00:00, 119.36it/s, loss=1.8714]


Test set: Accuracy = 57.7%, Mean loss = 1.8648


Epoch 3: 100%|██████████| 938/938 [00:08<00:00, 115.62it/s, loss=1.5297]


Test set: Accuracy = 60.6%, Mean loss = 1.4957


Epoch 4: 100%|██████████| 938/938 [00:08<00:00, 114.35it/s, loss=1.2789]


Test set: Accuracy = 63.1%, Mean loss = 1.2393


Epoch 5: 100%|██████████| 938/938 [00:07<00:00, 117.69it/s, loss=1.1274]


Test set: Accuracy = 64.5%, Mean loss = 1.0807
